In [1]:
import pandas as pd 
import numpy as np
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

In [2]:
df = pd.read_csv('../data/zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [8]:
# Function to turn dataset from Wide Format to Long Format and return only the state Nevada 
def melt_data_nv(df):
    melted = pd.melt(df, id_vars=['SizeRank','RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted.set_index('time',inplace=True)
    return melted[melted['State'] == 'NV']

In [9]:
# Turning the function into Nevada Dataframe
df_nv = melt_data_nv(df)
df_nv.head()

,SizeRank,RegionID,RegionName,City,State,Metro,CountyName,value
time,,,,,,,,
1996-04-01,116,95804,89108,Las Vegas,NV,Las Vegas,Clark,102500.0
1996-04-01,174,95817,89121,Las Vegas,NV,Las Vegas,Clark,106800.0
1996-04-01,196,95813,89117,Las Vegas,NV,Las Vegas,Clark,165100.0
1996-04-01,263,95785,89052,Henderson,NV,Las Vegas,Clark,185700.0
1996-04-01,310,95819,89123,Las Vegas,NV,Las Vegas,Clark,144000.0


In [15]:
len(list(set(df_nv.RegionID)))

103

In [10]:
df_time_series = pd.DataFrame(index=pd.to_datetime(df.columns[7:]), data=np.ones(len(df.columns)-7))
for i in range(df.shape[0]):
    df_time_series[df['RegionID'][i]] = df.iloc[i,7:]
df_time_series.drop(df_time_series.columns[0],axis=1, inplace=True)
df_time_series.fillna(method='bfill', inplace=True)

,84654,90668,91982,84616,93144,91733,61807,84640,91940,97564,...,59187,94711,62556,99032,62697,58333,59107,75672,93733,95851
1996-04-01,334200.0,235700.0,210400.0,498100.0,77300.0,95000.0,152900.0,216500.0,95400.0,766000.0,...,80800.0,135900.0,78300.0,136200.0,62500.0,94600.0,92700.0,57100.0,191100.0,176400.0
1996-05-01,335400.0,236900.0,212200.0,500900.0,77300.0,95200.0,152700.0,216700.0,95600.0,771100.0,...,80100.0,136300.0,78300.0,136600.0,62600.0,94300.0,92500.0,57300.0,192400.0,176300.0
1996-06-01,336500.0,236700.0,212200.0,503100.0,77300.0,95400.0,152600.0,216900.0,95800.0,776500.0,...,79400.0,136600.0,78200.0,136800.0,62700.0,94000.0,92400.0,57500.0,193700.0,176100.0
1996-07-01,337600.0,235400.0,210700.0,504600.0,77300.0,95700.0,152400.0,217000.0,96100.0,781900.0,...,78600.0,136900.0,78200.0,136800.0,62700.0,93700.0,92200.0,57700.0,195000.0,176000.0
1996-08-01,338500.0,233300.0,208300.0,505500.0,77400.0,95900.0,152300.0,217100.0,96400.0,787300.0,...,77900.0,137100.0,78100.0,136700.0,62700.0,93400.0,92100.0,58000.0,196300.0,175900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-01,1018700.0,316600.0,321200.0,1299000.0,120300.0,162800.0,414300.0,777900.0,172300.0,3778700.0,...,123400.0,257600.0,171300.0,341000.0,122800.0,216400.0,213100.0,130600.0,694700.0,348900.0
2018-01-01,1024400.0,318100.0,321200.0,1302700.0,120300.0,162800.0,413900.0,778500.0,173300.0,3770800.0,...,124400.0,258000.0,172400.0,342300.0,123200.0,213100.0,213700.0,131700.0,706400.0,350400.0
2018-02-01,1030700.0,319600.0,323000.0,1306400.0,120500.0,162900.0,411400.0,780500.0,174200.0,3763100.0,...,125500.0,260600.0,173600.0,345000.0,123200.0,209800.0,218300.0,132500.0,705300.0,353000.0
2018-03-01,1033800.0,321100.0,326900.0,1308500.0,121000.0,163500.0,413200.0,782800.0,175400.0,3779800.0,...,126600.0,264700.0,175800.0,348000.0,120700.0,209200.0,222700.0,133000.0,681500.0,356000.0


In [17]:
df_time_series = df_time_series[list(set(df_nv.RegionID))]
df_time_series

,95744,95750,95751,95753,95754,95760,95766,95768,95769,95770,...,95956,95957,95963,95966,399665,399666,399671,399672,399673,399674
1996-04-01,154600.0,131700.0,151000.0,137600.0,111500.0,153200.0,134100.0,131200.0,62100.0,122800.0,...,140200.0,110300.0,221900.0,233100.0,132500.0,196000.0,134000.0,175700.0,128400.0,264000.0
1996-05-01,154500.0,131600.0,151500.0,137600.0,111500.0,153700.0,133600.0,131000.0,62100.0,122800.0,...,140200.0,110500.0,221900.0,233100.0,132800.0,196000.0,134100.0,176400.0,129300.0,265200.0
1996-06-01,154300.0,131700.0,152000.0,137500.0,111400.0,154100.0,133000.0,130800.0,62100.0,122700.0,...,140100.0,110700.0,221900.0,233100.0,133000.0,195900.0,134300.0,177000.0,130200.0,266300.0
1996-07-01,154100.0,131700.0,152500.0,137400.0,111500.0,154400.0,132500.0,130600.0,62100.0,122700.0,...,140000.0,110900.0,221900.0,233100.0,133300.0,195700.0,134500.0,177700.0,131100.0,267300.0
1996-08-01,153900.0,131900.0,152900.0,137400.0,111500.0,154700.0,132000.0,130400.0,62100.0,122700.0,...,140100.0,111100.0,221900.0,233100.0,133600.0,195400.0,134900.0,178300.0,132000.0,268400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-01,300900.0,254200.0,331600.0,277000.0,240000.0,298300.0,227200.0,178400.0,140100.0,231600.0,...,302600.0,258200.0,237800.0,234100.0,292000.0,311700.0,262600.0,402200.0,290600.0,526400.0
2018-01-01,304400.0,257100.0,335100.0,280300.0,243200.0,299900.0,229500.0,179400.0,141600.0,234600.0,...,308700.0,259300.0,239900.0,236200.0,294800.0,315500.0,265500.0,406300.0,292200.0,527100.0
2018-02-01,307500.0,259500.0,338000.0,283100.0,246100.0,302500.0,231800.0,179800.0,145100.0,237200.0,...,314200.0,261500.0,241400.0,238100.0,297200.0,319500.0,268200.0,410300.0,293500.0,529700.0
2018-03-01,310200.0,261800.0,340800.0,285900.0,248800.0,305700.0,234200.0,180500.0,149100.0,239800.0,...,317100.0,265800.0,242800.0,240300.0,299500.0,323600.0,270500.0,416000.0,296600.0,538600.0


The model below iterates over every zipcode in Denver and predecits the prices in May 2018. It cross validates the model fitted with 17 years of training data, one year of validation set (horizon) and the model makes predictions per zipcode for each year between 2013 and 2017. It returns average Mean Absolute Percentage Error (MAPE) for each zipcode.

In [18]:
zipcodes = list(df_time_series.columns)
dict_mape = {}
dict_pred = {}
i = 1
for zipcode in zipcodes:
    # Create a new dataframe with dates and prices
    ts = pd.DataFrame({'ds':df_time_series.index, 'y':df_time_series[zipcode]})
    # Create a Prophet instance and fit the model
    m = Prophet(yearly_seasonality=False, changepoint_range=1, changepoint_prior_scale=0.5)
    m.fit(ts)

    # Create a dataframe holding dates for our forecast of one month period
    future = m.make_future_dataframe(periods=1, freq='M')
    # Create a forecast object which will hold all of the resulting data from the future.
    forecast = m.predict(future)

    # Fit the model with 17 years of training data, a horizon of 365 days (one year) and forecasting per year between 2013 and 2017
    df_cv = cross_validation(m, initial='6205 days', period='365 days', horizon = '365 days')
    df_p = performance_metrics(df_cv)

    # Create a dictionary with average MAPE of every zipcode
    dict_mape[zipcode] = np.mean(df_p['mape'])
    # Create a dictionary with forecasted prices of zipcodes for May 2018
    dict_pred[zipcode] = np.array(forecast.yhat)[-1]
    print(f'iteration={i}')
    i += 1

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:47<00:00,  9.41s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


iteration=1


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:29<00:00,  5.86s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


iteration=2


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:25<00:00,  5.09s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

iteration=101


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:27<00:00,  5.56s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


iteration=102


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:26<00:00,  5.38s/it]


iteration=103


In [19]:
dict_mape

{95744: 0.022498635550434295,
 95750: 0.04278110779000683,
 95751: 0.03408125028112888,
 95753: 0.02982231867033438,
 95754: 0.020673062452361255,
 95760: 0.021685927237354685,
 95766: 0.023356007127800827,
 95768: 0.0280397544224704,
 95769: 0.03602921269009029,
 95770: 0.03434730300348244,
 95771: 0.045912141431496335,
 95775: 0.052278179111354565,
 95779: 0.026860553573338542,
 95783: 0.04679328785772354,
 95785: 0.025238772814148578,
 95787: 0.05044588420751083,
 95788: 0.05167060208025199,
 95790: 0.031614012629685995,
 95792: 0.04179429582249042,
 95793: 0.02960237024440737,
 95794: 0.03774310893184554,
 95795: 0.022623376705750278,
 95798: 0.040796909553926605,
 95799: 0.023349681918025767,
 95800: 0.03320603155912621,
 95803: 0.05935506051263894,
 95804: 0.053552150532358844,
 95805: 0.07727488712982523,
 95806: 0.04725188507969076,
 95809: 0.043625364435827534,
 95811: 0.04808106594745356,
 95813: 0.029426083389244503,
 95814: 0.030271458421119,
 95815: 0.026782273794856256,
 

In [27]:
np.mean(list(dict_mape.values()))

0.037575237987934114

In [28]:
dict_pred

{95744: 315770.5862729762,
 95750: 268191.1951533978,
 95751: 346791.0020159719,
 95753: 290775.76827606466,
 95754: 255266.9013634051,
 95760: 310127.47890419036,
 95766: 240189.23249319484,
 95768: 182509.39476487262,
 95769: 153247.78988254067,
 95770: 246300.94475486007,
 95771: 234114.60925612983,
 95775: 208033.3162393082,
 95779: 357295.44824271597,
 95783: 200705.8126463924,
 95785: 421068.9254136304,
 95787: 154590.7171150974,
 95788: 219499.46418643033,
 95790: 316779.48254232004,
 95792: 255788.3319140967,
 95793: 299204.5957428111,
 95794: 328288.87104909273,
 95795: 279634.99856473086,
 95798: 220765.39635029866,
 95799: 254729.35021436046,
 95800: 212632.1144190938,
 95803: 198917.1941191062,
 95804: 212876.99612153973,
 95805: 337940.7916967204,
 95806: 200482.01372467002,
 95809: 309582.4499731394,
 95811: 180066.93932910392,
 95813: 344299.1564222108,
 95814: 271179.49889021384,
 95815: 241955.26361875163,
 95816: 267252.62681162066,
 95817: 212357.25585671476,
 95818:

In [30]:
zipcodes = list(df_time_series.columns)
dict_mape5 = {}
dict_pred5 = {}
i = 1
for zipcode in zipcodes:
    # Create a new dataframe with dates and prices
    ts = pd.DataFrame({'ds':df_time_series.index, 'y':df_time_series[zipcode]})
    # Create a Prophet instance and fit the model
    m = Prophet(yearly_seasonality=False, changepoint_range=0.9, changepoint_prior_scale=0.25)
    m.fit(ts)

    # Create a dataframe holding dates for our forecast of one month period
    future = m.make_future_dataframe(periods=1, freq='M')
    # Create a forecast object which will hold all of the resulting data from the future.
    forecast = m.predict(future)

    # Fit the model with 17 years of training data, a horizon of 365 days (one year) and forecasting per year between 2013 and 2017
    df_cv = cross_validation(m, initial='6205 days', period='365 days', horizon = '365 days')
    df_p = performance_metrics(df_cv)

    # Create a dictionary with average MAPE of every zipcode
    dict_mape5[zipcode] = np.mean(df_p['mape'])
    # Create a dictionary with forecasted prices of zipcodes for May 2018
    dict_pred5[zipcode] = np.array(forecast.yhat)[-1]
    print(f'iteration={i}')
    i += 1

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:27<00:00,  5.42s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


iteration=1


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:25<00:00,  5.01s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


iteration=2


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:19<00:00,  3.87s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

iteration=101


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:20<00:00,  4.03s/it]
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


iteration=102


INFO:fbprophet:Making 5 forecasts with cutoffs between 2013-04-02 00:00:00 and 2017-04-01 00:00:00
100%|██████████| 5/5 [00:21<00:00,  4.24s/it]

iteration=103


In [31]:
np.mean(list(dict_mape5.values()))

0.056519773001086626